In [ ]:
import dask
from dask.distributed import Client, LocalCluster

In [ ]:
import logging
logging.basicConfig(filename="logs.log", level=logging.INFO)
import os.path

import gc
import pickle
import os
import itertools
import MDAnalysis as mda
import MDAnalysis.transformations as trans
from MDAnalysis.analysis import align, pca, rms

#import pmda.rms
import nglview as nl
import numpy as np
import pandas as pd
import pickle
import dask.dataframe as dd

In [ ]:
from ENPMDA import MDDataFrame
from ENPMDA.preprocessing import TrajectoryEnsemble
from ENPMDA.utils import natural_keys

In [ ]:
cluster = LocalCluster(n_workers=64,
                       scheduler_port=8789,
                       memory_limit='4GB')

In [ ]:
client = Client(cluster)

In [ ]:
print(f"Current no. workers: {len(client.scheduler_info()['workers'])}")

In [ ]:
trajectory_list = []
topology_list = []
bonded_topology_list = []

In [ ]:
traj_note_dic = production_dic

for traj_note, load_location in zip(
    traj_note_dic["traj_note"], traj_note_dic["load_location"]
):
    for seed in sorted(os.listdir(load_location), key=natural_keys):
        if seed.startswith("SEEDS"):
            if os.path.exists(load_location + "/" + seed + "/" + 'md.xtc'):
                if not os.path.exists(load_location + "/" + seed + "/" + 'ca.pdb'):
                    raise ValueError('should have ca.pdb')
                trajectory_list.append(load_location + "/" + seed + "/md.xtc")
                topology_list.append(load_location + "/" + seed + "/ca.pdb")
                bonded_topology_list.append(load_location + "/" + seed + "/md.tpr")

In [ ]:
trajectory_ensemble = TrajectoryEnsemble(ensemble_name='msm_ensemble',
                                         trajectory_list=trajectory_list,
                                         topology_list=topology_list,
                                         bonded_topology_list=bonded_topology_list,
                                         skip=5,
                                         updating=False,
                                         only_raw=False)
trajectory_ensemble.load_ensemble()

In [ ]:
print(f"Number or trajectories in the ensemble is {len(trajectory_list)}")

In [ ]:
from ENPMDA.analysis import get_backbonetorsion, get_rmsd_init, get_atomic_position
from msm_a7_nachrs.prep.feature_selection import (
                    get_domain_position, get_domain_interdistance,
                    get_domain_intradistance,
                    get_pore_hydration, get_c_alpha_distance_10A,
                    get_c_alpha_distance_10A_2diff,
                    get_pore_hydration_prime,
                    get_membrane_thickness
                    )

In [ ]:
try:
    md_dataframe = MDDataFrame.load_dataframe('./a7_apos_feature/a7_apos_feature_md_dataframe')
except FileNotFoundError:
    md_dataframe = MDDataFrame(dataframe_name='a7_apos_feature')
    md_dataframe.add_traj_ensemble(trajectory_ensemble,
                               npartitions=244,
                               stride=2)

In [ ]:
print(f"Number of frames in the dataframe is {len(md_dataframe.dataframe)}")
print(f"dt in the dataframe is {md_dataframe.dataframe.traj_time.diff()[1] /1000} ns")
print(f"Number of systems in the dataframe is {len(md_dataframe.dataframe.system.unique())}")
print(f"Existing features are {md_dataframe.dataframe.columns}")

In [ ]:
md_dataframe.add_analysis(get_pore_hydration_prime)

In [ ]:
md_dataframe.add_analysis(get_c_alpha_distance_10A_2diff)

In [ ]:
md_dataframe.add_analysis(get_c_alpha_distance_10A)

In [ ]:
md_dataframe.add_analysis(get_domain_interdistance)

In [ ]:
md_dataframe.add_analysis(get_rmsd_init)
md_dataframe.add_analysis(get_backbonetorsion)
# md_dataframe.add_analysis(get_atomic_position)
#md_dataframe.add_analysis(get_domain_position)
md_dataframe.add_analysis(get_domain_interdistance)
md_dataframe.add_analysis(get_domain_intradistance)
md_dataframe.add_analysis(get_pore_hydration)

In [ ]:
u_ref = pickle.loads(open(md_dataframe.dataframe.universe_system[0], 'rb').read())

In [ ]:
md_dataframe.add_analysis(get_membrane_thickness)

In [ ]:
md_dataframe.sort_analysis_result()

In [ ]:
md_dataframe.dataframe['seed'] = md_dataframe.dataframe.traj_name.apply(lambda x: eval(x.split('/')[-3].split('_')[-1]))
md_dataframe.dataframe['pathway'] = md_dataframe.dataframe.traj_name.apply(lambda x: x.split('/')[-4])
md_dataframe.dataframe['ensemble'] = md_dataframe.dataframe.traj_name.apply(lambda x: x.split('/')[-5])

In [ ]:
md_dataframe.save('a7_apos_feature')

## Load the dataframe

In [ ]:
md_dataframe_new = MDDataFrame.load_dataframe('./a7_apos_feature/a7_apos_feature_md_dataframe')

In [ ]:
#md_dataframe_new.sort_analysis_result()

In [ ]:
feature_dataframe = md_dataframe_new.get_feature([
                        'ca_distance_10A',
                        ])

In [ ]:
md_dataframe_new.get_feature_info('ca_distance_10A')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(data=feature_dataframe,
             x='system',
             y='pore_hydration')
plt.show()